In [1]:
import numpy as np
import pandas as pd
import pickle
import csv
from os import listdir
import keras as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, LSTM, Embedding, TimeDistributed,BatchNormalization, Dense,Dropout, RepeatVector, Activation, Flatten
from keras.preprocessing import image, sequence
from keras.layers.wrappers import Bidirectional
from keras.applications import VGG16
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
import scipy
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ModelCheckpoint
from matplotlib.pyplot import imshow
from keras.preprocessing.text import Tokenizer
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
embedding_dim = 128

In [ ]:
"""total_string = "<sss> " + row[4] + " . " + row[5] + " " + row[6]"""

In [20]:
all_caption_list = []

with open('preprocess.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        row_insert = []
        row_insert.append(row[0])
        total_string = "<sss> " + row[5] + " " + row[6] + " <eee>"
        row_insert.append(total_string)
        all_caption_list.append(row_insert)

with open("train_caption_final.csv", "w", newline='') as output:
    writer = csv.writer(output)
    writer.writerows(all_caption_list)

In [3]:
encoded_images = pickle.load( open( "visual_features.pickle", "rb" ) )

In [5]:
#find vocab_size
#find total_sample
def initialize():
        df = pd.read_csv('train_caption_final.csv', delimiter=',')
        nb_samples = df.shape[0]
        iteration = df.iterrows()
        caps = []
        for i in range(nb_samples):
            x = iteration.__next__()
            caps.append(x[1][1])

        total_samples=0
        for text in caps:
            total_samples+=len(text.split())-1
        print("Total samples : "+str(total_samples))
        
        words = [txt.split() for txt in caps]
        unique = []
        for word in words:
            unique.extend(word)
        unique = list(set(unique))
        vocab_size = len(unique)
        word_index = {}
        index_word = {}
        for i, word in enumerate(unique):
            word_index[word]=i
            index_word[i]=word

        max_len = 0
        for caption in caps:
            if(len(caption.split()) > max_len):
                max_len = len(caption.split())
        max_cap_len = max_len
        print("Vocabulary size: "+str(vocab_size))
        print("Maximum caption length: "+str(max_cap_len))
        
        return max_cap_len, vocab_size, total_samples, word_index, index_word


In [6]:
max_capt_len, vocab_size, total_samples, word_to_index, index_to_word = initialize()

Total samples : 119082
Vocabulary size: 3395
Maximum caption length: 177


In [ ]:
#Bakwaas
def linear_model_combined(optimizer='Adadelta'):    

    # declare input
    inlayer =Input(shape=(2048,1))
    flatten = Flatten()(inlayer)

    modela = Dense(1024)(flatten)
    modela = Activation('relu')(modela)
    modela = Dense(512)(modela)

    modelb = Dense(1024)(flatten)
    modelb = Activation('relu')(modelb)
    modelb = Dense(512)(modelb)

    model_concat = concatenate([modela, modelb])


    model_concat = Activation('relu')(model_concat)
    model_concat = Dense(256)(model_concat)
    model_concat = Activation('relu')(model_concat)

    model_concat = Dense(4)(model_concat)
    model_concat = Activation('softmax')(model_concat)

    model_combined = Model(inputs=inlayer,outputs=model_concat)

    model_combined.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model_combined

In [ ]:
"""EMBEDDING_DIM = 300

image_model = Sequential()
image_model.add(Dense(EMBEDDING_DIM, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_capt_len))

   
lang_model = Sequential()
lang_model.add(Embedding(vocab_size,EMBEDDING_DIM , input_length=max_capt_len))
lang_model.add(Bidirectional(LSTM(256,return_sequences=True)))
lang_model.add(Dropout(0.5))
lang_model.add(BatchNormalization())
lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))

   
fin_model = Sequential()
#fin_model.add(Merge([image_model, lang_model], mode='concat'))
fin_model.add(K.layers.concatenate([image_model, lang_model],axis=1))
fin_model.add(Dropout(0.5))
fin_model.add(BatchNormalization())
fin_model.add(Bidirectional(LSTM(1000,return_sequences=False)))

fin_model.add(Dense(vocab_size))
fin_model.add(Activation('softmax'))
print ("Model created!")"""

In [ ]:
"""fin_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
image_model.summary()
lang_model.summary()
fin_model.summary()
"""

In [7]:
def create_model(return_model = False):
        
        image_decod_input = Input(shape=(2048,))
        image_decod = Dense(embedding_dim, input_dim=2048, activation='relu')(image_decod_input)
        image_decod = RepeatVector(max_capt_len)(image_decod)
        
        
        capt_decod_input = Input(shape=(max_capt_len,))
        capt_decod = Embedding(vocab_size, 256, input_length=max_capt_len)(capt_decod_input)
        capt_decod = LSTM(256,return_sequences=True)(capt_decod)
        capt_decod = TimeDistributed(Dense(embedding_dim))(capt_decod)
        
        model = K.layers.concatenate([image_decod, capt_decod])
        model = LSTM(1000,return_sequences=False)(model)
        model = Dense(vocab_size)(model)
        prediction = Activation('softmax')(model)
        
        model = Model(inputs=[image_decod_input, capt_decod_input], outputs=prediction)

        if(return_model==True):
            return model

        model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
        return model

In [8]:
def generate_a_batch(batch_size = 32):
        partial_caps = []
        next_words = []
        images = []
        gen_count = 0
        df = pd.read_csv('train_caption_final.csv', delimiter=',')
        nb_samples = df.shape[0]
        iter = df.iterrows()
        caps = []
        imgs = []
        for i in range(nb_samples):
            x = iter.__next__()
            caps.append(x[1][1])
            imgs.append(x[1][0])


        total_count = 0
        while True:
            image_counter = -1
            for text in caps:
                image_counter+=1
                current_image = encoded_images[imgs[image_counter]][0]
                #print(current_image.shape)
                for i in range(len(text.split())-1):
                    total_count+=1
                    partial = [word_to_index[txt] for txt in text.split()[:i+1]]
                    partial_caps.append(partial)
                    next = np.zeros(vocab_size)
                    next[word_to_index[text.split()[i+1]]] = 1
                    next_words.append(next)
                    images.append(current_image)
                    #print(images[0].shape)

                    if total_count>=batch_size:
                        next_words = np.asarray(next_words)
                        images = np.asarray(images)
                        partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_capt_len, padding='post')
                        total_count = 0
                        gen_count+=1
                        if gen_count%10 == 0:
                            print("batch count: "+str(gen_count))
                        yield [[images, partial_caps], next_words]
                        partial_caps = []
                        next_words = []
                        images = []

In [9]:
batch_size = 16
epochs = 10

In [ ]:
image_caption_model = create_model()
file_name = 'saved_weights_epoch_{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(file_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
checkpoints_list = [checkpoint]
image_caption_model.fit_generator(generate_a_batch(batch_size=batch_size), steps_per_epoch=total_samples/batch_size, epochs=epochs, verbose=2, callbacks=checkpoints_list)

Epoch 1/10
batch count: 10
batch count: 20
batch count: 30
batch count: 40
batch count: 50
batch count: 60
batch count: 70
batch count: 80
batch count: 90
batch count: 100
batch count: 110
batch count: 120
batch count: 130
batch count: 140
batch count: 150
batch count: 160
batch count: 170
batch count: 180
batch count: 190
batch count: 200
batch count: 210
batch count: 220
batch count: 230
batch count: 240
batch count: 250
batch count: 260
batch count: 270
batch count: 280
batch count: 290
batch count: 300
batch count: 310
batch count: 320
batch count: 330
batch count: 340
batch count: 350
batch count: 360
batch count: 370
batch count: 380
batch count: 390
batch count: 400
batch count: 410
batch count: 420
batch count: 430
batch count: 440
batch count: 450
batch count: 460
batch count: 470
batch count: 480
batch count: 490
batch count: 500
batch count: 510
batch count: 520
batch count: 530
batch count: 540
batch count: 550
batch count: 560
batch count: 570
batch count: 580
batch count:

batch count: 4620
batch count: 4630
batch count: 4640
batch count: 4650
batch count: 4660
batch count: 4670
batch count: 4680
batch count: 4690
batch count: 4700
batch count: 4710
batch count: 4720
batch count: 4730
batch count: 4740
batch count: 4750
batch count: 4760
batch count: 4770
batch count: 4780
batch count: 4790
batch count: 4800
batch count: 4810
batch count: 4820
batch count: 4830
batch count: 4840
batch count: 4850
batch count: 4860
batch count: 4870
batch count: 4880
batch count: 4890
batch count: 4900
batch count: 4910
batch count: 4920
batch count: 4930
batch count: 4940
batch count: 4950
batch count: 4960
batch count: 4970
batch count: 4980
batch count: 4990
batch count: 5000
batch count: 5010
batch count: 5020
batch count: 5030
batch count: 5040
batch count: 5050
batch count: 5060
batch count: 5070
batch count: 5080
batch count: 5090
batch count: 5100
batch count: 5110
batch count: 5120
batch count: 5130
batch count: 5140
batch count: 5150
batch count: 5160
batch coun

batch count: 9100
batch count: 9110
batch count: 9120
batch count: 9130
batch count: 9140
batch count: 9150
batch count: 9160
batch count: 9170
batch count: 9180
batch count: 9190
batch count: 9200
batch count: 9210
batch count: 9220
batch count: 9230
batch count: 9240
batch count: 9250
batch count: 9260
batch count: 9270
batch count: 9280
batch count: 9290
batch count: 9300
batch count: 9310
batch count: 9320
batch count: 9330
batch count: 9340
batch count: 9350
batch count: 9360
batch count: 9370
batch count: 9380
batch count: 9390
batch count: 9400
batch count: 9410
batch count: 9420
batch count: 9430
batch count: 9440
batch count: 9450
batch count: 9460
batch count: 9470
batch count: 9480
batch count: 9490
batch count: 9500
batch count: 9510
batch count: 9520
batch count: 9530
batch count: 9540
batch count: 9550
batch count: 9560
batch count: 9570
batch count: 9580
batch count: 9590
batch count: 9600
batch count: 9610
batch count: 9620
batch count: 9630
batch count: 9640
batch coun

batch count: 13460
batch count: 13470
batch count: 13480
batch count: 13490
batch count: 13500
batch count: 13510
batch count: 13520
batch count: 13530
batch count: 13540
batch count: 13550
batch count: 13560
batch count: 13570
batch count: 13580
batch count: 13590
batch count: 13600
batch count: 13610
batch count: 13620
batch count: 13630
batch count: 13640
batch count: 13650
batch count: 13660
batch count: 13670
batch count: 13680
batch count: 13690
batch count: 13700
batch count: 13710
batch count: 13720
batch count: 13730
batch count: 13740
batch count: 13750
batch count: 13760
batch count: 13770
batch count: 13780
batch count: 13790
batch count: 13800
batch count: 13810
batch count: 13820
batch count: 13830
batch count: 13840
batch count: 13850
batch count: 13860
batch count: 13870
batch count: 13880
batch count: 13890
batch count: 13900
batch count: 13910
batch count: 13920
batch count: 13930
batch count: 13940
batch count: 13950
batch count: 13960
batch count: 13970
batch count:

batch count: 17700
batch count: 17710
batch count: 17720
batch count: 17730
batch count: 17740
batch count: 17750
batch count: 17760
batch count: 17770
batch count: 17780
batch count: 17790
batch count: 17800
batch count: 17810
batch count: 17820
batch count: 17830
batch count: 17840
batch count: 17850
batch count: 17860
batch count: 17870
batch count: 17880
batch count: 17890
batch count: 17900
batch count: 17910
batch count: 17920
batch count: 17930
batch count: 17940
batch count: 17950
batch count: 17960
batch count: 17970
batch count: 17980
batch count: 17990
batch count: 18000
batch count: 18010
batch count: 18020
batch count: 18030
batch count: 18040
batch count: 18050
batch count: 18060
batch count: 18070
batch count: 18080
batch count: 18090
batch count: 18100
batch count: 18110
batch count: 18120
batch count: 18130
batch count: 18140
batch count: 18150
batch count: 18160
batch count: 18170
batch count: 18180
batch count: 18190
batch count: 18200
batch count: 18210
batch count:

batch count: 22020
batch count: 22030
batch count: 22040
batch count: 22050
batch count: 22060
batch count: 22070
batch count: 22080
batch count: 22090
batch count: 22100
batch count: 22110
batch count: 22120
batch count: 22130
batch count: 22140
batch count: 22150
batch count: 22160
batch count: 22170
batch count: 22180
batch count: 22190
batch count: 22200
batch count: 22210
batch count: 22220
batch count: 22230
batch count: 22240
batch count: 22250
batch count: 22260
batch count: 22270
batch count: 22280
batch count: 22290
batch count: 22300
batch count: 22310
batch count: 22320
batch count: 22330
Epoch 00003: loss improved from 5.83669 to 5.83669, saving model to saved_weights_epoch_03.hdf5
 - 22258s - loss: 5.8367 - acc: 0.0359
Epoch 4/10
batch count: 22340
batch count: 22350
batch count: 22360
batch count: 22370
batch count: 22380
batch count: 22390
batch count: 22400
batch count: 22410
batch count: 22420
batch count: 22430
batch count: 22440
batch count: 22450
batch count: 22460